# Building a Simple ReAct Agent from Scratch

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
# Import necessary libraries and modules
import openai
import re
import httpx
import os

In [3]:
from openai import OpenAI
client = OpenAI()

MODEL = 'gpt-5-mini'
prompt = 'Write something short but funny.'

chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [4]:
chat_completion.choices[0].message.content

'Parallel lines have so much in common — it’s a shame they’ll never meet.'

## Creating the Agent Class

In [5]:
class Agent:
  def __init__(self, system = ''):
    self.system = system
    self.messages = []

    if self.system:
      self.messages.append({"role": "system", "content": system})

  def __call__(self, prompt):
    self.messages.append({'role': 'user', 'content': prompt})
    result = self.execute()

    self.messages.append({'role': 'assistant', 'content': result})
    return result
  
  def execute(self, model='gpt-5-mini', temperature=0):
    chat_completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=self.messages
    )
    return chat_completion.choices[0].message.content

## Creating the ReAct Prompt

In [ ]:
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will give you the result of running those actions.

Your available actions are:

calculate:
Runs a calculation and returns the number - uses Python to be sure to use Floating point syntax if necessary.

get_cost:
e.g get_cost: book
Returns the cost of a book

wikipedia:
e.g wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up to Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does the pen cost?
Thought: I need to find out the cost of a pen using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5

Example session #2:

Question: What is the capital of France?
Thought: I need to look up the capital of France on Wikipedia
Action: wikipedia: capital of France
PAUSE

You will be called again with this:

Observation: The capital of France is Paris.

You then output:

Answer: The capital of France is Paris.
'''.strip()

## Creating the Tools

In [ ]:
def calculate(what):
  return eval(what)

def get_cost(item):
  if item in 'pen':
    return 'A pen costs $5'
  elif item in 'book':
    return 'A book costs $15'
  elif item in 'laptop':
    return 'A laptop costs $1500'
  else:
    return 'Item not found'
